# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
0.983033,"""12249502927619…","""86042211074412…","""../../../../da…","""16070643130240…","""12581602152447…",false
0.144221,"""78538896275013…","""14790277734238…","""../../../../da…","""16070643130240…","""17360215677611…",true
0.679745,"""12500012403984…","""15012080527241…","""../../../../da…","""41393944907364…","""12581602152447…",false
1.0,"""78538896275013…","""11144430626797…","""../../../../da…","""28532511430333…","""12581602152447…",false
0.718982,"""12249502927619…","""16643628383471…","""../../../../da…","""28532511430333…","""17933557521831…",true
0.0,"""12500012403984…","""98098447921227…","""../../../../da…","""11521820815398…",null,false
0.02333,"""12500012403984…","""28113095851923…","""../../../../da…","""28532511430333…","""17933557521831…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""94913893337741…","""59833892884998…","""../../../../da…","""FirstMortgage3…","""61333049257759…","""Rejected"""
0.144221,"""58997686691872…","""13697290440563…","""../../../../da…","""FirstMortgage3…","""12719367232162…","""Accepted"""
0.679745,"""27620955018113…","""16872435357674…","""../../../../da…","""MoneyMarketSav…","""61333049257759…","""Rejected"""
1.0,"""58997686691872…","""17242578652642…","""../../../../da…","""BasicChecking""","""61333049257759…","""Rejected"""
0.718982,"""94913893337741…","""13881469920525…","""../../../../da…","""BasicChecking""","""92392120807932…","""Accepted"""
0.0,"""27620955018113…","""51564708334291…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""27620955018113…","""86567181109056…","""../../../../da…","""BasicChecking""","""92392120807932…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""33381313484202…","""26451989467737…","""../../../../da…","""FirstMortgage3…","""11731520622551…","""Rejected"""
0.144221,"""70159638173123…","""41603067539608…","""../../../../da…","""FirstMortgage3…","""14231758363267…","""Accepted"""
0.679745,"""60423976143506…","""17540059047875…","""../../../../da…","""MoneyMarketSav…","""11731520622551…","""Rejected"""
1.0,"""70159638173123…","""12440572819090…","""../../../../da…","""BasicChecking""","""11731520622551…","""Rejected"""
0.718982,"""33381313484202…","""10090786266790…","""../../../../da…","""BasicChecking""","""11500124902462…","""Accepted"""
0.0,"""60423976143506…","""15823557688274…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""60423976143506…","""78927259699167…","""../../../../da…","""BasicChecking""","""11500124902462…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
Customer_City=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
0.983033,"""18304550312009…","""70010494637149…","""FirstMortgage3…","""33221983541513…","""Rejected"""
0.144221,"""33584084860306…","""16613078792937…","""FirstMortgage3…","""76463602502463…","""Accepted"""
0.679745,"""10056018342410…","""11120597934597…","""MoneyMarketSav…","""33221983541513…","""Rejected"""
1.0,"""33584084860306…","""13794431511254…","""BasicChecking""","""33221983541513…","""Rejected"""
0.718982,"""18304550312009…","""46986485332567…","""BasicChecking""","""12359858489125…","""Accepted"""
0.0,"""10056018342410…","""43192427758124…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""10056018342410…","""90619178897974…","""BasicChecking""","""12359858489125…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""af486bc1815c72…","""c73023dda51de1…","""../../../../da…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
0.144221,"""3cdf7b1ae1342e…","""5414b197fb87b3…","""../../../../da…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
0.679745,"""806ef34eb8237f…","""892f5d61a9215e…","""../../../../da…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
1.0,"""3cdf7b1ae1342e…","""c3aa807d9e927a…","""../../../../da…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
0.718982,"""af486bc1815c72…","""c2d492f9a05073…","""../../../../da…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
0.0,"""806ef34eb8237f…","""b21059221605c4…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""806ef34eb8237f…","""80cbe489dcecd4…","""../../../../da…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
